In [71]:
import pandas as pd
from utils.func import remove_outliers, price_per_square_meter_per_postal_code

In [72]:
train = pd.read_json('train.json')
test = pd.read_json('test.json')

combined_df = pd.concat([train, test], axis=0, ignore_index=True)


In [73]:
train = train.convert_dtypes()



In [74]:
from pandas import CategoricalDtype

kit_cat= CategoricalDtype(categories=['not installed', 'usa uninstalled', 'semi equipped', 'usa semi equipped', 'hyper equipped', 'usa hyper equipped', 'installed', 'usa installed'], ordered=True)
train["Kitchen"] = train["Kitchen"].fillna("not installed")
train["Kitchen"] = train["Kitchen"].astype(kit_cat).cat.codes

building_state_type = pd.CategoricalDtype(
    categories=[
        "not known",
        "to be done up",
        "to restore",
        "to renovate",
        "just renovated",
        "good",
        "as new",
    ],
    ordered=True,
)
train["StateOfBuilding"] = train["StateOfBuilding"].fillna("not known")
train["StateOfBuilding"] = (
    train["StateOfBuilding"].astype(building_state_type).cat.codes
)

heating_cat= CategoricalDtype(categories=['not known', 'fueloil', 'gas','carbon', 'wood', 'pellet', 'electric',  'solar'], ordered=True)
train["Heating"] = train["Heating"].fillna("not known")
train["Heating"] = (
    train["Heating"].astype(heating_cat).cat.codes
)

train["SubtypeOfProperty"] = train["SubtypeOfProperty"].fillna("not known")
train["SubtypeOfProperty"] = train["SubtypeOfProperty"].astype("category").cat.codes

# train = pd.get_dummies(train, columns=['SubtypeOfProperty'])

In [75]:
train = remove_outliers(train, 'Price', 0.05, 0.95)

train = price_per_square_meter_per_postal_code(train)

train.dropna(subset=['PricePerSquareMeter'],axis=0)

train = remove_outliers(train, 'PricePerSquareMeter', 0.05, 0.95)

e:\BeCode\AI\immo-eliza-train-test-alexandre\utils\func.py:25: RuntimeWarning: divide by zero encountered in scalar divide
  lambda x: x["Price"].mean() / (x["SurfaceOfGood"] + x["LivingArea"]).mean()


In [76]:
train['SurfaceOfGood'] = train['SurfaceOfGood'].fillna(train['LivingArea'])
train = train.drop(columns=[
    'Url', 
    # 'SubtypeOfProperty',
    'PropertyId', 
    # 'Kitchen', 
    # 'StateOfBuilding', 
    # 'Heating',
    'GardenArea', 
    # 'NumberOfFacades', 
    'ConstructionYear',
    # 'LivingArea',
    'Openfire',
    # 'Garden',
    # 'Terrace',
    # 'Furnished'
])

In [77]:
train = train.dropna()
print(train.columns)
print(train.shape)
train.to_csv('train.csv', index=False)

Index(['Furnished', 'Terrace', 'Garden', 'TypeOfProperty', 'PostalCode',
       'SubtypeOfProperty', 'TypeOfSale', 'Price', 'Kitchen',
       'StateOfBuilding', 'Heating', 'Bedrooms', 'SurfaceOfGood',
       'SwimmingPool', 'NumberOfFacades', 'LivingArea', 'PricePerSquareMeter'],
      dtype='object')
(14682, 17)
